In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import GrubbsTest

In [ ]:
def remove_ev_submeter_in_pair(
    ami_df_i, 
    pct_0_thresh_main=0.1, 
    pct_0_thresh_subm=0.6, 
    enforce_corr=True, 
    corr_thresh=0.5, 
    #remove_undetermined=True, 
    value_col='value', 
    time_col='index', 
    PN_col='aep_premise_nb', 
    SN_col='serialnumber'
):
    r"""
    Given a pair of meter connected to a single premise, this tries to determine which is the EV submeter in the pair, 
      and keeps that which is not.
    Basically, the submeter only monitors the EV, whereas the other monitors the usage of the entire premise.
    Thus, the EV signal should be 0 for a majority of the time, and the two meters should be correlated.
    
    This is a somewhat specialized function for use with engineering the EV dataset
    
    ami_df_i:
        An AMI DF containing data for a single premise which has two meters
        
    pct_0_thresh_main:
        The maximum allowed percentage of 0 values for a meter to be considered the main meter
    pct_0_thresh_subm:
        The minimum allowed percentage of 0 values for a meter to be considered the submeter
    """
    #-------------------------
    assert(ami_df_i[PN_col].nunique()==1)
    assert(pct_0_thresh_main < pct_0_thresh_subm)
    #-------------------------
    SNs_i = ami_df_i[SN_col].unique().tolist()
    assert(len(SNs_i)<=2)
    #-------------------------
    if len(SNs_i)==1:
        return ami_df_i
    #-------------------------
    ami_df_i_1 = ami_df_i[ami_df_i[SN_col]==SNs_i[0]]
    ami_df_i_2 = ami_df_i[ami_df_i[SN_col]==SNs_i[1]]
    if time_col=='index':
        ami_df_i_1 = ami_df_i_1.sort_index()
        ami_df_i_2 = ami_df_i_2.sort_index()
    else:
        ami_df_i_1 = ami_df_i_1.sort_values(by=[time_col])
        ami_df_i_2 = ami_df_i_2.sort_values(by=[time_col])
    #--------------------------------------------------
    # NOTE: I have seen some SNs which have a bunch of 0.002 values, which, for the purposes here should be treated as 0s
    #       This is why I use <0.005 instead of ==0 and >=0.005 instead of !=0 below
    #--------------------------------------------------
    pct_1 = (ami_df_i_1[value_col]<0.005).sum()/ami_df_i_1.shape[0]
    pct_2 = (ami_df_i_2[value_col]<0.005).sum()/ami_df_i_2.shape[0]
    #-------------------------
    if pct_1 <= pct_0_thresh_main:
        defn_1 = 1
    elif pct_1 >= pct_0_thresh_subm:
        defn_1 = 0
    else:
        defn_1 = -1
    #-----
    if pct_2 <= pct_0_thresh_main:
        defn_2 = 1
    elif pct_2 >= pct_0_thresh_subm:
        defn_2 = 0
    else:
        defn_2 = -1
    #-------------------------
    if defn_1+defn_2==1:
        pass_pcts=True
    else:
        pass_pcts=False
    #-------------------------
    if not pass_pcts:
        return
    
    #--------------------------------------------------
    pass_corr=True
    if enforce_corr:
        if time_col=='index':
            ami_df_i_12 = pd.merge(ami_df_i_1[value_col], ami_df_i_2[value_col], left_index=True, right_index=True, how='outer')
        else:
            ami_df_i_12 = pd.merge(ami_df_i_1[value_col], ami_df_i_2[value_col], left_on=time_col, right_on=time_col, how='outer')
        # For the correlation only using values which are non-zero (and aligning, meaning non-NaN)
        ami_df_i_12 = ami_df_i_12[(ami_df_i_12[f'{value_col}_x']>=0.005) & (ami_df_i_12[f'{value_col}_y']>=0.005)]
        corr_val = ami_df_i_12[f'{value_col}_x'].corr(ami_df_i_12[f'{value_col}_y'])
        if corr_val >= corr_thresh:
            pass_corr = True
        else:
            pass_corr = False
            
    #--------------------------------------------------
    if not (pass_pcts and pass_corr):
        return
    #-------------------------
    if defn_1==1:
        assert(defn_2==0) # Sanity check, not needed
        return ami_df_i[ami_df_i[SN_col]==SNs_i[0]]
    else:
        assert(defn_1==0 and defn_2==1) # Sanity check, not needed
        return ami_df_i[ami_df_i[SN_col]==SNs_i[1]]

In [ ]:
def select_pairs_w_submeter(
    ami_df_i, 
    pct_0_thresh_main=0.1, 
    pct_0_thresh_subm=0.6, 
    enforce_corr=True, 
    corr_thresh=0.5, 
    #remove_undetermined=True, 
    value_col='value', 
    time_col='index', 
    PN_col='aep_premise_nb', 
    SN_col='serialnumber'
):
    r"""
    Given a pair of meter connected to a single premise, this tries to determine which is the EV submeter in the pair, 
      and keeps that which is not.
    Basically, the submeter only monitors the EV, whereas the other monitors the usage of the entire premise.
    Thus, the EV signal should be 0 for a majority of the time, and the two meters should be correlated.
    
    This is a somewhat specialized function for use with engineering the EV dataset
    
    ami_df_i:
        An AMI DF containing data for a single premise which has two meters
        
    pct_0_thresh_main:
        The maximum allowed percentage of 0 values for a meter to be considered the main meter
    pct_0_thresh_subm:
        The minimum allowed percentage of 0 values for a meter to be considered the submeter
    """
    #-------------------------
    assert(ami_df_i[PN_col].nunique()==1)
    assert(pct_0_thresh_main < pct_0_thresh_subm)
    #-------------------------
    SNs_i = ami_df_i[SN_col].unique().tolist()
    assert(len(SNs_i)<=2)
    #-------------------------
    if len(SNs_i)==1:
        return 
    #-------------------------
    ami_df_i_1 = ami_df_i[ami_df_i[SN_col]==SNs_i[0]]
    ami_df_i_2 = ami_df_i[ami_df_i[SN_col]==SNs_i[1]]
    if time_col=='index':
        ami_df_i_1 = ami_df_i_1.sort_index()
        ami_df_i_2 = ami_df_i_2.sort_index()
    else:
        ami_df_i_1 = ami_df_i_1.sort_values(by=[time_col])
        ami_df_i_2 = ami_df_i_2.sort_values(by=[time_col])
    #--------------------------------------------------
    # NOTE: I have seen some SNs which have a bunch of 0.002 values, which, for the purposes here should be treated as 0s
    #       This is why I use <0.005 instead of ==0 and >=0.005 instead of !=0 below
    #--------------------------------------------------
    pct_1 = (ami_df_i_1[value_col]<0.005).sum()/ami_df_i_1.shape[0]
    pct_2 = (ami_df_i_2[value_col]<0.005).sum()/ami_df_i_2.shape[0]
    #-------------------------
    if pct_1 <= pct_0_thresh_main:
        defn_1 = 1
    elif pct_1 >= pct_0_thresh_subm:
        defn_1 = 0
    else:
        defn_1 = -1
    #-----
    if pct_2 <= pct_0_thresh_main:
        defn_2 = 1
    elif pct_2 >= pct_0_thresh_subm:
        defn_2 = 0
    else:
        defn_2 = -1
    #-------------------------
    if defn_1+defn_2==1:
        pass_pcts=True
    else:
        pass_pcts=False
    #-------------------------
    if not pass_pcts:
        return
    
    #--------------------------------------------------
    pass_corr=True
    if enforce_corr:
        if time_col=='index':
            ami_df_i_12 = pd.merge(ami_df_i_1[value_col], ami_df_i_2[value_col], left_index=True, right_index=True, how='outer')
        else:
            ami_df_i_12 = pd.merge(ami_df_i_1[value_col], ami_df_i_2[value_col], left_on=time_col, right_on=time_col, how='outer')
        # For the correlation only using values which are non-zero (and aligning, meaning non-NaN)
        ami_df_i_12 = ami_df_i_12[(ami_df_i_12[f'{value_col}_x']>=0.005) & (ami_df_i_12[f'{value_col}_y']>=0.005)]
        corr_val = ami_df_i_12[f'{value_col}_x'].corr(ami_df_i_12[f'{value_col}_y'])
        if corr_val >= corr_thresh:
            pass_corr = True
        else:
            pass_corr = False
            
    #--------------------------------------------------
    if not (pass_pcts and pass_corr):
        return
    #-------------------------
    if defn_1==1:
        assert(defn_2==0) # Sanity check, not needed
        ami_df_i_main = ami_df_i[ami_df_i[SN_col]==SNs_i[0]].copy()
        ami_df_i_subm = ami_df_i[ami_df_i[SN_col]==SNs_i[1]].copy()
    else:
        assert(defn_1==0 and defn_2==1) # Sanity check, not needed
        ami_df_i_main =  ami_df_i[ami_df_i[SN_col]==SNs_i[1]].copy()
        ami_df_i_subm =  ami_df_i[ami_df_i[SN_col]==SNs_i[0]].copy()
    #-------------------------
    if time_col=='index':
        return_ami_df_i = pd.merge(
            ami_df_i_main, 
            ami_df_i_subm[[value_col, SN_col]], 
            left_index=True, 
            right_index=True, 
            how='inner'
        )
    else:
        return_ami_df_i = pd.merge(
            ami_df_i_main, 
            ami_df_i_subm[[value_col, time_col, SN_col]], 
            left_on=time_col, 
            right_on=time_col, 
            how='inner'
        )
    #----------
    return_ami_df_i = return_ami_df_i.rename(columns={
        f'{value_col}_x': f'{value_col}_main', 
        f'{value_col}_y': f'{value_col}_subm', 
        f'{SN_col}_x': f'{SN_col}_main', 
        f'{SN_col}_y': f'{SN_col}_subm', 
    })
    #----------
    return_ami_df_i[f'{value_col}_delt'] = return_ami_df_i[f'{value_col}_main']-return_ami_df_i[f'{value_col}_subm']
    #-------------------------
    return return_ami_df_i

In [ ]:
build_ami_df = False

In [ ]:
if build_ami_df:
    ami_df = GenAn.read_df_from_csv_dir_batches(
        files_dir=r'C:\Users\s346557\Documents\LocalData\EVs\Data', 
        file_path_glob=r'*.csv'
    )
    #-------------------------
    ami_df = ami_df[
        (ami_df['aep_derived_uom']=='KWH') & 
        (ami_df['aep_srvc_qlty_idntfr']=='TOTAL')
    ].copy()
    #-----
    ami_df['timezoneoffset'] = ami_df['starttimeperiod'].str[-6:]
    #-------------------------
    ami_df = AMINonVee.perform_std_initiation_and_cleaning(
        ami_df, 
        timestamp_col=None
    )
    #-----
    ami_df = Utilities_dt.strip_tz_info_and_convert_to_dt(
        df=ami_df, 
        time_col='starttimeperiod', 
        placement_col='starttimeperiod_local', 
        run_quick=True, 
        n_strip=6, 
        inplace=False
    )
    ami_df = Utilities_dt.strip_tz_info_and_convert_to_dt(
        df=ami_df, 
        time_col='endtimeperiod', 
        placement_col='endtimeperiod_local', 
        run_quick=True, 
        n_strip=6, 
        inplace=False
    )
    ami_df=ami_df.set_index('starttimeperiod_local', drop=False)
    #-------------------------
    ami_df.to_pickle(r'C:\Users\s346557\Documents\LocalData\EVs\ami_df.pkl')
else:
    ami_df = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\EVs\ami_df.pkl')
#-------------------------
all_trff_dfs = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\EVs\all_trff_dfs.pkl')

In [ ]:
ami_df_full = ami_df.copy()

In [ ]:
ami_df['aep_premise_nb'].nunique()

In [ ]:
all_trff_dfs['PREM_NB'].nunique()

In [ ]:
all_trff_dfs_found     = all_trff_dfs[all_trff_dfs['PREM_NB'].isin(ami_df['aep_premise_nb'].unique().tolist())]
all_trff_dfs_not_found = all_trff_dfs[~all_trff_dfs['PREM_NB'].isin(ami_df['aep_premise_nb'].unique().tolist())]

In [ ]:
print(all_trff_dfs_found['PREM_NB'].nunique())
print(all_trff_dfs_not_found['PREM_NB'].nunique())

In [ ]:
print(ami_df['aep_premise_nb'].nunique())
print(ami_df['serialnumber'].nunique())

In [ ]:
evs_prems = all_trff_dfs[all_trff_dfs['EV']==1]['PREM_NB'].unique().tolist()
non_prems = all_trff_dfs[all_trff_dfs['EV']==0]['PREM_NB'].unique().tolist()

# aep_premise_nb in ami_df is of type object (i.e., a string), whereas PREM_NB in trff_df is int64
evs_prems = [str(x) for x in evs_prems]
non_prems = [str(x) for x in non_prems]
#-----
ami_df_evs = ami_df[ami_df['aep_premise_nb'].isin(evs_prems)].copy()
ami_df_non = ami_df[ami_df['aep_premise_nb'].isin(non_prems)].copy()
#----
assert(ami_df.shape[0]==ami_df_evs.shape[0]+ami_df_non.shape[0])

In [ ]:
ami_df = ami_df_evs.groupby('aep_premise_nb', as_index=False, group_keys=False).apply(
    lambda x: select_pairs_w_submeter(
        ami_df_i=x, 
        pct_0_thresh_main=0.1, 
        pct_0_thresh_subm=0.6, 
        enforce_corr=True, 
        corr_thresh=0.5, 
        value_col='value', 
        time_col='index', 
        PN_col='aep_premise_nb', 
        SN_col='serialnumber'
    )
)

In [ ]:
print(ami_df['aep_premise_nb'].nunique())
# print(ami_df['serialnumber'].nunique())

In [ ]:
ami_df = ami_df.sort_index()
ami_df['date'] = ami_df.index
#-------------------------
ami_df_main = ami_df[['aep_premise_nb', 'serialnumber_main', 'value_main', 'date']].copy()
ami_df_delt = ami_df[['aep_premise_nb', 'serialnumber_main', 'value_delt', 'date']].copy()
ami_df_subm = ami_df[['aep_premise_nb', 'serialnumber_subm', 'value_subm', 'date']].copy()
#-------------------------
ami_df_main = ami_df_main.rename(columns={
    'serialnumber_main':'serialnumber', 
    'value_main':'value'
})

ami_df_delt = ami_df_delt.rename(columns={
    'serialnumber_main':'serialnumber', 
    'value_delt':'value'
})

ami_df_subm = ami_df_subm.rename(columns={
    'serialnumber_subm':'serialnumber', 
    'value_subm':'value'
})

# SEE WEBPAGES
https://stackoverflow.com/questions/22583391/peak-signal-detection-in-realtime-timeseries-data
https://stackoverflow.com/questions/22583391/peak-signal-detection-in-realtime-timeseries-data/43512887#43512887
https://stackoverflow.com/questions/22583391/peak-signal-detection-in-realtime-timeseries-data/56451135#56451135

In [ ]:
# class real_time_peak_detection():
#     def __init__(self, array, lag, threshold, influence):
#         self.y = list(array)
#         self.length = len(self.y)
#         self.lag = lag
#         self.threshold = threshold
#         self.influence = influence
#         self.signals = [0] * len(self.y)
#         self.filteredY = np.array(self.y).tolist()
#         self.avgFilter = [0] * len(self.y)
#         self.stdFilter = [0] * len(self.y)
#         self.avgFilter[self.lag - 1] = np.mean(self.y[0:self.lag]).tolist()
#         self.stdFilter[self.lag - 1] = np.std(self.y[0:self.lag]).tolist()

#     def thresholding_algo(self, new_value):
#         self.y.append(new_value)
#         i = len(self.y) - 1
#         self.length = len(self.y)
#         if i < self.lag:
#             return 0
#         elif i == self.lag:
#             self.signals = [0] * len(self.y)
#             self.filteredY = np.array(self.y).tolist()
#             self.avgFilter = [0] * len(self.y)
#             self.stdFilter = [0] * len(self.y)
#             self.avgFilter[self.lag] = np.mean(self.y[0:self.lag]).tolist()
#             self.stdFilter[self.lag] = np.std(self.y[0:self.lag]).tolist()
#             return 0

#         self.signals += [0]
#         self.filteredY += [0]
#         self.avgFilter += [0]
#         self.stdFilter += [0]

#         if abs(self.y[i] - self.avgFilter[i - 1]) > (self.threshold * self.stdFilter[i - 1]):

#             if self.y[i] > self.avgFilter[i - 1]:
#                 self.signals[i] = 1
#             else:
#                 self.signals[i] = -1

#             self.filteredY[i] = self.influence * self.y[i] + \
#                 (1 - self.influence) * self.filteredY[i - 1]
#             self.avgFilter[i] = np.mean(self.filteredY[(i - self.lag):i])
#             self.stdFilter[i] = np.std(self.filteredY[(i - self.lag):i])
#         else:
#             self.signals[i] = 0
#             self.filteredY[i] = self.y[i]
#             self.avgFilter[i] = np.mean(self.filteredY[(i - self.lag):i])
#             self.stdFilter[i] = np.std(self.filteredY[(i - self.lag):i])

#         return self.signals[i]


def thresholding_algo_OLD(
    y, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0 #DOESN'T DO ANYTHING IN THIS FUNCTION
):
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    for i in range(lag, len(y)):
        if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter [i-1]:
            if y[i] > avgFilter[i-1]:
                signals[i] = 1
            else:
                signals[i] = -1

            filteredY[i] = influence * y[i] + (1 - influence) * filteredY[i-1]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])
        else:
            signals[i] = 0
            filteredY[i] = y[i]
            avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
            stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))


def thresholding_algo(
    y, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    #-----
    if len(y) < lag:
        return dict(signals = np.asarray(np.zeros(len(y))),
                    avgFilter = np.asarray(np.mean(y)),
                    stdFilter = np.asarray(np.std(y)))
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    non_signal_Y = []
    for i in range(lag, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = 0
            filteredY[i] = y[i]
            non_signal_Y.append(y[i])    
        else:
            if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter[i-1]:
                if y[i] > avgFilter[i-1]:
                    signals[i] = 1
                else:
                    signals[i] = -1
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    filteredY[i] = influence * y[i] + (1 - influence) * np.mean(non_signal_Y)
            else:
                signals[i] = 0
                filteredY[i] = y[i]
                non_signal_Y.append(y[i])
        avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
        stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))

In [ ]:
def thresholding_algo_median(
    y, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    #-----
    if len(y) < lag:
        return dict(signals = np.asarray(np.zeros(len(y))),
                    avgFilter = np.asarray(np.median(y)),
                    stdFilter = np.asarray(np.std(y)))
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.median(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    non_signal_Y = []
    for i in range(lag, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = 0
            filteredY[i] = y[i]
            non_signal_Y.append(y[i])    
        else:
            if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter[i-1]:
                if y[i] > avgFilter[i-1]:
                    signals[i] = 1
                else:
                    signals[i] = -1
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    filteredY[i] = influence * y[i] + (1 - influence) * np.median(non_signal_Y)
            else:
                signals[i] = 0
                filteredY[i] = y[i]
                non_signal_Y.append(y[i])
        avgFilter[i] = np.median(filteredY[(i-lag+1):i+1])
        stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))

In [ ]:
def mad(y):
    return np.mean(np.abs(y - np.mean(y)))

def thresholding_algo_mad(
    y, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    #-----
    if len(y) < lag:
        return dict(signals = np.asarray(np.zeros(len(y))),
                    avgFilter = np.asarray(np.mean(y)),
                    stdFilter = np.asarray(mad(y)))
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = mad(y[0:lag])
    non_signal_Y = []
    for i in range(lag, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = 0
            filteredY[i] = y[i]
            non_signal_Y.append(y[i])    
        else:
            if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter[i-1]:
                if y[i] > avgFilter[i-1]:
                    signals[i] = 1
                else:
                    signals[i] = -1
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    filteredY[i] = influence * y[i] + (1 - influence) * np.mean(non_signal_Y)
            else:
                signals[i] = 0
                filteredY[i] = y[i]
                non_signal_Y.append(y[i])
        avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
        stdFilter[i] = mad(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))


def thresholding_algo_median_mad(
    y, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    #-----
    if len(y) < lag:
        return dict(signals = np.asarray(np.zeros(len(y))),
                    avgFilter = np.asarray(np.median(y)),
                    stdFilter = np.asarray(mad(y)))
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.median(y[0:lag])
    stdFilter[lag - 1] = mad(y[0:lag])
    non_signal_Y = []
    for i in range(lag, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = 0
            filteredY[i] = y[i]
            non_signal_Y.append(y[i])    
        else:
            if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter[i-1]:
                if y[i] > avgFilter[i-1]:
                    signals[i] = 1
                else:
                    signals[i] = -1
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    filteredY[i] = influence * y[i] + (1 - influence) * np.median(non_signal_Y)
            else:
                signals[i] = 0
                filteredY[i] = y[i]
                non_signal_Y.append(y[i])
        avgFilter[i] = np.median(filteredY[(i-lag+1):i+1])
        stdFilter[i] = mad(filteredY[(i-lag+1):i+1])

    return dict(signals = np.asarray(signals),
                avgFilter = np.asarray(avgFilter),
                stdFilter = np.asarray(stdFilter))

In [ ]:
ami_df = ami_df.sort_index()

In [ ]:
tmp_df = ami_df_main[ami_df_main['aep_premise_nb']=='020093989'].sort_index().copy()

rtpd = thresholding_algo(
    y=tmp_df['value'].tolist(), 
    lag=48,
    threshold=10, 
    influence=0.0, 
    signal_abs_threshold=1.0
)
tmp_df['signals'] = rtpd['signals']
#-------------------------
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='date', y='value', hue='serialnumber', data=tmp_df)
sns.scatterplot(ax=ax, x='date', y='value', hue='serialnumber', data=tmp_df[tmp_df['signals']==1], palette='hls')
ax.grid(True, which='both')

In [ ]:
tmp_df = ami_df_subm[ami_df_subm['aep_premise_nb']=='020093989'].sort_index().copy()

rtpd = thresholding_algo(
    y=tmp_df['value'].tolist(), 
    lag=48,
    threshold=10, 
    influence=0.0, 
    signal_abs_threshold=1.0
)
tmp_df['signals'] = rtpd['signals']
#-------------------------
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='date', y='value', hue='serialnumber', data=tmp_df)
sns.scatterplot(ax=ax, x='date', y='value', hue='serialnumber', data=tmp_df[tmp_df['signals']==1], palette='hls')
ax.grid(True, which='both')

In [ ]:
tmp_df = ami_df_delt[ami_df_delt['aep_premise_nb']=='020093989'].sort_index().copy()

rtpd = thresholding_algo(
    y=tmp_df['value'].tolist(), 
    lag=48,
    threshold=10, 
    influence=0.0, 
    signal_abs_threshold=1.0
)
tmp_df['signals'] = rtpd['signals']
#-------------------------
fig, ax = Plot_General.default_subplots()
sns.lineplot(ax=ax, x='date', y='value', hue='serialnumber', data=tmp_df)
sns.scatterplot(ax=ax, x='date', y='value', hue='serialnumber', data=tmp_df[tmp_df['signals']==1], palette='hls')
ax.grid(True, which='both')

In [ ]:
def set_signal_groups_in_df(
    df_i, 
    lag, 
    threshold,
    influence, 
    signal_abs_threshold, 
    value_col='mean_TRS value', 
    SN_col='serialnumber',
    signal_col='signals', 
    return_signal_group_col='signal_grp'
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    rtpd_i = thresholding_algo(
        y=df_i[value_col].tolist(), 
        lag=lag,
        threshold=threshold, 
        influence=influence, 
        signal_abs_threshold=signal_abs_threshold
    )
    df_i[signal_col] = rtpd_i['signals']    
    
    #-------------------------
    signals_df_i = df_i.reset_index()[df_i.reset_index()[signal_col]==1]
    #-----
    tmp_signal_grp_col = Utilities.generate_random_string()
    signals_df_i[tmp_signal_grp_col] = np.nan
    #-----
    tmp_idx_col = Utilities.generate_random_string()
    signals_df_i[tmp_idx_col] = signals_df_i.index
    #-----
    signals_df_i[tmp_signal_grp_col] = signals_df_i[tmp_idx_col].diff().ne(1).cumsum()
    #-------------------------
    return_df = df_i.copy()
    return_df[return_signal_group_col] = np.nan
    return_df.iloc[signals_df_i.index,-1] = signals_df_i[tmp_signal_grp_col]
    #-------------------------
    return return_df

In [ ]:
def build_df_i_signals_gpd(
    df_i, 
    lag, 
    threshold,
    influence, 
    signal_abs_threshold, 
    value_col='mean_TRS value', 
    SN_col='serialnumber', 
    time_col='starttimeperiod_local', 
    signal_col='signals'
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    assert(df_i.index.name == time_col)
    df_i = df_i.sort_index()
    #-------------------------
    signal_group_col='signal_grp'
    df_i = set_signal_groups_in_df(
        df_i=df_i, 
        lag=lag, 
        threshold=threshold,
        influence=influence, 
        signal_abs_threshold=signal_abs_threshold, 
        value_col=value_col, 
        SN_col=SN_col,
        signal_col=signal_col, 
        return_signal_group_col=signal_group_col
    )
    #-------------------------
    df_i_signals = df_i[df_i[signal_group_col].notna()]
    df_i_signals = df_i_signals.drop(columns=[signal_col])
    df_i_signals[signal_group_col] = df_i_signals[signal_group_col].astype(int)
    df_i_signals = df_i_signals.reset_index()
    assert(time_col in df_i_signals.columns.tolist())
    #-------------------------
    # I think I like the mean of the max, time width of the peak, and the spacing of the peaks
    # For the width of the peak, there needs to be a more sophisticated approach.
    #   e.g., if the peak is one data point, the width is 0
    #   For now, however, I will simply take max-min

    df_i_signals_gpd = df_i_signals.groupby([signal_group_col]).agg({
        time_col:['mean', 'std', 'min', 'max'], 
        value_col:['mean', 'std', 'max']
    })
    df_i_signals_gpd=df_i_signals_gpd.sort_values(by=[(time_col, 'mean')])
    df_i_signals_gpd[(time_col, 'max_m_min')] = df_i_signals_gpd[(time_col, 'max')] - df_i_signals_gpd[(time_col, 'min')]
    #-------------------------
    return df_i_signals_gpd

In [ ]:
def extract_features_for_SN(
    df_i, 
    lag, 
    threshold,
    influence, 
    signal_abs_threshold, 
    value_col='mean_TRS value', 
    SN_col='serialnumber', 
    time_col='starttimeperiod_local', 
    signal_col='signals'
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    # I think I like the mean of the max, time width of the peak, and the spacing of the peaks
    # For the width of the peak, there needs to be a more sophisticated approach.
    #   e.g., if the peak is one data point, the width is 0
    #   For now, however, I will simply take max-min
    
    df_i_signals_gpd = build_df_i_signals_gpd(
        df_i=df_i, 
        lag=lag, 
        threshold=threshold,
        influence=influence, 
        signal_abs_threshold=signal_abs_threshold, 
        value_col=value_col, 
        SN_col=SN_col, 
        time_col=time_col, 
        signal_col=signal_col
    )
    #-------------------------
    peak_max_mean     = df_i_signals_gpd[(value_col, 'max')].mean()
    peak_width_mean   = df_i_signals_gpd[(time_col, 'max_m_min')].mean()
    peak_spacing_mean = df_i_signals_gpd[(time_col, 'mean')].diff().mean()
    # Below, the date being used is of no matter, any random date works, it's simply
    #   to make pd.to_datetime happy
    if df_i_signals_gpd.shape[0]>0:
        peak_hour_mean    = pd.to_datetime(
            '2023-01-01 ' + df_i_signals_gpd[(time_col, 'mean')].dt.strftime('%H:%M:%S'), 
            format="%Y-%m-%d %H:%M:%S"
        ).mean().round('H').time().hour
    else:
        peak_hour_mean = np.nan
    #-------------------------
    features_srs = pd.Series({
        'peak_max_mean':     peak_max_mean, 
        'peak_width_mean':   peak_width_mean, 
        'peak_spacing_mean': peak_spacing_mean, 
        'peak_hour_mean':    peak_hour_mean, 
    })
    features_srs.name = df_i[SN_col].unique()[0]
    return features_srs

In [ ]:
def extract_features_for_SN_v2(
    df_i, 
    lag, 
    threshold,
    influence, 
    signal_abs_threshold, 
    value_col='mean_TRS value', 
    SN_col='serialnumber', 
    time_col='starttimeperiod_local', 
    signal_col='signals'
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    assert(df_i.index.name == time_col)
    df_i = df_i.sort_index()
    #-------------------------
    signal_group_col='signal_grp'
    df_i = find_signals_and_set_signal_groups_in_df_i(
        df_i=df_i, 
        lag=lag, 
        threshold=threshold,
        influence=influence, 
        signal_abs_threshold=signal_abs_threshold, 
        value_col=value_col, 
        SN_col=SN_col,
        signal_col=signal_col, 
        return_signal_group_col=signal_group_col
    )
    #-------------------------
    df_i_signals = df_i[df_i[signal_group_col].notna()]
    df_i_signals = df_i_signals.drop(columns=[signal_col])
    df_i_signals[signal_group_col] = df_i_signals[signal_group_col].astype(int)
    #-----   
    drop_idx = False
    if df_i_signals.index.name in df_i_signals.columns:
        drop_idx = True
    df_i_signals = df_i_signals.reset_index(drop=drop_idx)
    assert(time_col in df_i_signals.columns.tolist())
    #-------------------------
    # Features split into:
    #   1. Using all data in peaks pooled together
    #   2. First grouping by signal group, then extracting features
    #-------------------------
    # Features (1):
    peak_mean = df_i_signals[value_col].mean()
    peak_std  = df_i_signals[value_col].std()
    #-------------------------
    # Features (2):
    #-----
    # I think I like the mean of the max, time width of the peak, and the spacing of the peaks
    # For the width of the peak, there needs to be a more sophisticated approach.
    #   e.g., if the peak is one data point, the width is 0
    #   For now, however, I will simply take max-min

    df_i_signals_gpd = df_i_signals.groupby([signal_group_col]).agg({
        time_col:['mean', 'std', 'min', 'max'], 
        value_col:['mean', 'std', 'max']
    })
    df_i_signals_gpd=df_i_signals_gpd.sort_values(by=[(time_col, 'mean')])
    df_i_signals_gpd[(time_col, 'max_m_min')] = df_i_signals_gpd[(time_col, 'max')] - df_i_signals_gpd[(time_col, 'min')]
    #-------------------------
    #-------------------------
    peak_max_mean     = df_i_signals_gpd[(value_col, 'max')].mean()
    peak_max_std      = df_i_signals_gpd[(value_col, 'max')].std()
    #-----
    peak_width_mean   = df_i_signals_gpd[(time_col, 'max_m_min')].mean()
    peak_width_std    = df_i_signals_gpd[(time_col, 'max_m_min')].std()
    #-----
    peak_spacing_mean = df_i_signals_gpd[(time_col, 'mean')].diff().mean()
    peak_spacing_std  = df_i_signals_gpd[(time_col, 'mean')].diff().std()
    #-----
    # Below, the date being used is of no matter, any random date works, it's simply
    #   to make pd.to_datetime happy
    if df_i_signals_gpd.shape[0]>0:
        peak_hour_mean    = pd.to_datetime(
            '2023-01-01 ' + df_i_signals_gpd[(time_col, 'mean')].dt.strftime('%H:%M:%S'), 
            format="%Y-%m-%d %H:%M:%S"
        ).mean().round('H').time().hour
    else:
        peak_hour_mean = np.nan
    #-------------------------
    features_srs = pd.Series({
        'peak_mean':         peak_mean, 
        'peak_std':          peak_std, 
        #-----
        'peak_max_mean':     peak_max_mean, 
        'peak_max_std':      peak_max_std, 
        #-----
        'peak_width_mean':   peak_width_mean, 
        'peak_width_std':    peak_width_std, 
        #-----
        'peak_spacing_mean': peak_spacing_mean,
        'peak_spacing_std':  peak_spacing_std,
        #-----
        'peak_hour_mean':    peak_hour_mean, 
    })
    features_srs.name = df_i[SN_col].unique()[0]
    return features_srs

In [ ]:
ami_df_main

In [ ]:
peak_df_evs = ami_df_main.groupby(['serialnumber']).apply(
    lambda x: extract_features_for_SN_v2(
        df_i=x, 
        lag=48, 
        threshold=10, 
        influence=0.0, 
        signal_abs_threshold=2.0, 
        value_col='value'
    )
)

peak_df_evs

In [ ]:
peak_df_non = ami_df_delt.groupby(['serialnumber']).apply(
    lambda x: extract_features_for_SN_v2(
        df_i=x, 
        lag=48, 
        threshold=10, 
        influence=0.0, 
        signal_abs_threshold=2.0, 
        value_col='value'
    )
)

peak_df_non

In [ ]:
peak_df_evs['target']=1
peak_df_non['target']=0

In [ ]:
peak_df=pd.concat([peak_df_evs, peak_df_non])
peak_df=peak_df.sample(frac=1)

In [ ]:
peak_df_OG = peak_df.copy()

In [ ]:
peak_df = peak_df_OG.copy()

In [ ]:
# peak_df['peak_max_mean']     = peak_df['peak_max_mean'].fillna(0)
# peak_df['peak_width_mean']   = peak_df['peak_width_mean'].fillna(pd.Timedelta.max)
# peak_df['peak_spacing_mean'] = peak_df['peak_spacing_mean'].fillna(pd.Timedelta.max)
# peak_df['peak_hour_mean']    = peak_df['peak_hour_mean'].fillna(-1)

peak_df['peak_mean']         = peak_df['peak_mean'].fillna(0)
peak_df['peak_std']          = peak_df['peak_std'].fillna(-1)
#-----
peak_df['peak_max_mean']     = peak_df['peak_max_mean'].fillna(0)
peak_df['peak_max_std']      = peak_df['peak_max_std'].fillna(-1)
#-----
peak_df['peak_width_mean']   = peak_df['peak_width_mean'].fillna(pd.Timedelta(0))
peak_df['peak_width_std']    = peak_df['peak_width_std'].fillna(pd.Timedelta(-1))
# peak_df['peak_width_std']    = peak_df['peak_width_std'].fillna(pd.Timedelta(0))
# peak_df['peak_width_std']    = peak_df['peak_width_std'].fillna(pd.Timedelta(pd.Timedelta.max))
#-----
peak_df['peak_spacing_mean'] = peak_df['peak_spacing_mean'].fillna(pd.Timedelta(0))
peak_df['peak_spacing_std']  = peak_df['peak_spacing_std'].fillna(pd.Timedelta(-1))
# peak_df['peak_spacing_std']  = peak_df['peak_spacing_std'].fillna(pd.Timedelta(0))
# peak_df['peak_spacing_std']  = peak_df['peak_spacing_std'].fillna(pd.Timedelta(pd.Timedelta.max))
#-----
peak_df['peak_hour_mean']    = peak_df['peak_hour_mean'].fillna(-1)

In [ ]:
peak_df['peak_width_mean']   = peak_df['peak_width_mean'].dt.total_seconds()/60
peak_df['peak_width_std']    = peak_df['peak_width_std'].dt.total_seconds()/60
#-----
peak_df['peak_spacing_mean'] = peak_df['peak_spacing_mean'].dt.total_seconds()/60
peak_df['peak_spacing_std']  = peak_df['peak_spacing_std'].dt.total_seconds()/60

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve

In [ ]:
# peak_df = peak_df.drop(columns=[
#     'peak_mean', 'peak_std', 
#     'peak_max_std', 
#     'peak_width_std', 
#     'peak_spacing_std'
# ])

In [ ]:
peak_df_train, peak_df_test = train_test_split(peak_df, test_size=0.33, random_state=42)

In [ ]:
X_train = peak_df_train[[x for x in peak_df_train.columns.tolist() if x!='target']]
y_train = peak_df_train['target']
#-----
X_test = peak_df_test[[x for x in peak_df_test.columns.tolist() if x!='target']]
y_test = peak_df_test['target']
#-----

In [ ]:
forest_clf = RandomForestClassifier(n_estimators = 10, max_depth=None, n_jobs=None)
start = time.time()
forest_clf.fit(X_train, y_train)
print(time.time()-start)

In [ ]:
y_pred_train = forest_clf.predict(X_train)
print('*****'*5)
print('TRAINING DATASET')
print('*****'*5)
print(f"#(target==1): {y_train.sum()}")
print(f"#(target==0): {y_train.shape[0]-y_train.sum()}")
print(f"%(target==1): {100*(y_train.sum()/(y_train.shape[0]))}")
print('-----'*5)
print("ACCURACY  OF THE MODEL: ", accuracy_score(y_train, y_pred_train))
print("PRECISION OF THE MODEL: ", precision_score(y_train, y_pred_train))
print("RECALL    OF THE MODEL: ", recall_score(y_train, y_pred_train))
print("F1        OF THE MODEL: ", f1_score(y_train, y_pred_train))
print()

y_pred = forest_clf.predict(X_test)
print('*****'*5)
print('TESTING DATASET')
print('*****'*5)
print(f"#(target==1): {y_test.sum()}")
print(f"#(target==0): {y_test.shape[0]-y_test.sum()}")
print(f"%(target==1): {100*(y_test.sum()/(y_test.shape[0]))}")
print('-----'*5)
print("ACCURACY  OF THE MODEL: ", accuracy_score(y_test, y_pred))
print("PRECISION OF THE MODEL: ", precision_score(y_test, y_pred))
print("RECALL    OF THE MODEL: ", recall_score(y_test, y_pred))
print("F1        OF THE MODEL: ", f1_score(y_test, y_pred))
print()


In [ ]:
cmd = ConfusionMatrixDisplay(
    confusion_matrix(y_train, y_pred_train, normalize=None), 
    display_labels=['Not EV','EV']
)
cmd.plot(values_format='.3e')
# cmd.plot(values_format='')
# cmd.ax_.set(xlabel='Predicted', ylabel='True')
cmd.ax_.set_xlabel('Predicted', fontsize=16)
cmd.ax_.set_ylabel('True', fontsize=16)
cmd.ax_.set_title('Train', fontsize=20, fontweight='semibold')
# cmd.figure_.set_size_inches(12, 10)

cmd.figure_.text(1.0, 0.9, "# Entries:    {:.4e}".format(y_train.shape[0]), fontsize=14)
cmd.figure_.text(1.0, 0.8, "# Outages:  {:.4e}".format(y_train.sum()), fontsize=14)
cmd.figure_.text(1.0, 0.7, "# Baseline:  {:.4e}".format(y_train.shape[0]-y_train.sum()), fontsize=14)
cmd.figure_.text(1.0, 0.6, "% Outages:  {:.4f}".format(100*y_train.sum()/y_train.shape[0]), fontsize=14)


cmd.figure_.text(1.0, 0.4, "ACCURACY:  {:.4f}".format(accuracy_score(y_train, y_pred_train)), fontsize=14)
cmd.figure_.text(1.0, 0.3, "PRECISION:  {:.4f}".format(precision_score(y_train, y_pred_train)), fontsize=14)
cmd.figure_.text(1.0, 0.2, "RECALL:       {:.4f}".format(recall_score(y_train, y_pred_train)), fontsize=14)


In [ ]:
cmd = ConfusionMatrixDisplay(
    confusion_matrix(y_test, y_pred, normalize=None), 
    display_labels=['Not EV','EV']
)
cmd.plot(values_format='.3e')
# cmd.plot(values_format='')
# cmd.ax_.set(xlabel='Predicted', ylabel='True')
cmd.ax_.set_xlabel('Predicted', fontsize=16)
cmd.ax_.set_ylabel('True', fontsize=16)
cmd.ax_.set_title('Test', fontsize=20, fontweight='semibold')
# cmd.figure_.set_size_inches(12, 10)

cmd.figure_.text(1.0, 0.9, "# Entries:    {:.4e}".format(y_test.shape[0]), fontsize=14)
cmd.figure_.text(1.0, 0.8, "# Outages:  {:.4e}".format(y_test.sum()), fontsize=14)
cmd.figure_.text(1.0, 0.7, "# Baseline:  {:.4e}".format(y_test.shape[0]-y_test.sum()), fontsize=14)
cmd.figure_.text(1.0, 0.6, "% Outages:  {:.4f}".format(100*y_test.sum()/y_test.shape[0]), fontsize=14)

cmd.figure_.text(1.0, 0.4, "ACCURACY:  {:.4f}".format(accuracy_score(y_test, y_pred)), fontsize=14)
cmd.figure_.text(1.0, 0.3, "PRECISION:  {:.4f}".format(precision_score(y_test, y_pred)), fontsize=14)
cmd.figure_.text(1.0, 0.2, "RECALL:       {:.4f}".format(recall_score(y_test, y_pred)), fontsize=14)

In [ ]:
assert(len(peak_df.columns[:-1])==len(forest_clf.feature_importances_))
importances = list(zip(peak_df.columns[:-1], forest_clf.feature_importances_))
importances_srtd = sorted(importances, key=lambda x: x[1], reverse=True)

In [ ]:
importances_srtd

In [ ]:
def make_it_funky(df_i, n_entries=500, val_col='value', SN_col='serialnumber'):
    if df_i.shape[0] < n_entries:
        return None
    assert(df_i[SN_col].nunique()==1)
    vals_i = df_i[val_col].sort_index().iloc[:n_entries].tolist()
    return_i = pd.Series(vals_i)
    return_i.name = df_i[SN_col].unique()[0]
    return return_i

In [ ]:
keras_df_evs = ami_df_evs.groupby(['serialnumber']).apply(
    lambda x: make_it_funky(x)
)
keras_df_non = ami_df_non.groupby(['serialnumber']).apply(
    lambda x: make_it_funky(x)
)

In [ ]:
keras_df_evs['target']=1
keras_df_non['target']=0

In [ ]:
keras_df = pd.concat([keras_df_evs, keras_df_non])

In [ ]:
keras_df

In [ ]:
keras_df=keras_df.dropna()

In [ ]:
keras_df

In [ ]:
keras_df_train, keras_df_test = train_test_split(keras_df, test_size=0.33, random_state=42)

In [ ]:
x_train = keras_df_train[[x for x in keras_df_train.columns.tolist() if x != 'target']].values
y_train = keras_df_train['target'].values
x_train=x_train.reshape((x_train.shape[0], x_train.shape[1], 1))

In [ ]:
x_test = keras_df_test[[x for x in keras_df_test.columns.tolist() if x != 'target']].values
y_test = keras_df_test['target'].values
x_test=x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [ ]:
# No longer needed because train_test_splot
# idx = np.random.permutation(len(x_train))
# x_train = x_train[idx]
# y_train = y_train[idx]

In [ ]:
y_train

In [ ]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def make_model(input_shape):
    num_classes=2
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

In [ ]:
model = make_model(input_shape=x_train.shape[1:])
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 500
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

In [ ]:
model = keras.models.load_model("best_model.h5")

test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test accuracy", test_acc)
print("Test loss", test_loss)

In [ ]:
train_loss, train_acc = model.evaluate(x_train, y_train)
print("Train accuracy", train_acc)
print("Train loss", train_loss)

In [ ]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
print("ACCURACY  OF THE MODEL: ", accuracy_score(y_test, y_pred))
print("PRECISION OF THE MODEL: ", precision_score(y_test, y_pred))
print("RECALL    OF THE MODEL: ", recall_score(y_test, y_pred))
print("F1        OF THE MODEL: ", f1_score(y_test, y_pred))

In [ ]:
ami_df_full

In [ ]:
nSNs_per_PN = ami_df_full[['aep_premise_nb', 'serialnumber']].drop_duplicates()['aep_premise_nb'].value_counts()

In [ ]:
nSNs_per_PN[nSNs_per_PN==1].index.tolist()

In [ ]:
ami_df_HOLDOUT = ami_df_full[ami_df_full['aep_premise_nb'].isin(nSNs_per_PN[nSNs_per_PN==1].index.tolist())]

In [ ]:
set(ami_df_HOLDOUT['aep_premise_nb'].tolist()).intersection(ami_df['aep_premise_nb'].tolist())

In [ ]:
keras_df_HOLDOUT = ami_df_HOLDOUT.groupby(['serialnumber', 'aep_premise_nb']).apply(
    lambda x: make_it_funky(x)
)

In [ ]:
keras_df_HOLDOUT['target']=np.nan

In [ ]:
keras_df_HOLDOUT

In [ ]:
keras_df_HOLDOUT.loc[keras_df_HOLDOUT.index.get_level_values(1).isin(evs_prems), 'target']=1
keras_df_HOLDOUT.loc[keras_df_HOLDOUT.index.get_level_values(1).isin(non_prems), 'target']=0
assert(pd.isna(keras_df_HOLDOUT['target']).sum()==0)

In [ ]:
keras_df_HOLDOUT=keras_df_HOLDOUT.dropna()

In [ ]:
x_HOLDOUT = keras_df_HOLDOUT[[x for x in keras_df_HOLDOUT.columns.tolist() if x != 'target']].values
y_HOLDOUT = keras_df_HOLDOUT['target'].values
x_HOLDOUT=x_HOLDOUT.reshape((x_HOLDOUT.shape[0], x_HOLDOUT.shape[1], 1))

In [ ]:
y_pred_HOLDOUT = model.predict(x_HOLDOUT)
y_pred_HOLDOUT = np.argmax(y_pred_HOLDOUT, axis=1)
print("ACCURACY  OF THE MODEL: ", accuracy_score(y_HOLDOUT, y_pred_HOLDOUT))
print("PRECISION OF THE MODEL: ", precision_score(y_HOLDOUT, y_pred_HOLDOUT))
print("RECALL    OF THE MODEL: ", recall_score(y_HOLDOUT, y_pred_HOLDOUT))
print("F1        OF THE MODEL: ", f1_score(y_HOLDOUT, y_pred_HOLDOUT))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
ami_df = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\EVs\ami_df.pkl')
all_trff_dfs = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\EVs\all_trff_dfs.pkl')

In [ ]:
evs_prems = all_trff_dfs[all_trff_dfs['EV']==1]['PREM_NB'].unique().tolist()
non_prems = all_trff_dfs[all_trff_dfs['EV']==0]['PREM_NB'].unique().tolist()

# aep_premise_nb in ami_df is of type object (i.e., a string), whereas PREM_NB in trff_df is int64
evs_prems = [str(x) for x in evs_prems]
non_prems = [str(x) for x in non_prems]
#-----
ami_df_evs = ami_df[ami_df['aep_premise_nb'].isin(evs_prems)].copy()
ami_df_non = ami_df[ami_df['aep_premise_nb'].isin(non_prems)].copy()
#----
assert(ami_df.shape[0]==ami_df_evs.shape[0]+ami_df_non.shape[0])

In [ ]:
def make_it_funky(df_i, n_entries=500, val_col='value', SN_col='serialnumber'):
    if df_i.shape[0] < n_entries:
        return None
    assert(df_i[SN_col].nunique()==1)
    vals_i = df_i[val_col].sort_index().iloc[:n_entries].tolist()
    return_i = pd.Series(vals_i)
    return_i.name = df_i[SN_col].unique()[0]
    return return_i

In [ ]:
keras_df_evs = ami_df_evs.groupby(['serialnumber']).apply(
    lambda x: make_it_funky(x)
)
keras_df_non = ami_df_non.groupby(['serialnumber']).apply(
    lambda x: make_it_funky(x)
)

In [ ]:
keras_df_evs['target']=1
keras_df_non['target']=0

In [ ]:
keras_df = pd.concat([keras_df_evs, keras_df_non])

In [ ]:
keras_df

In [ ]:
keras_df=keras_df.dropna()

In [ ]:
keras_df

In [ ]:
keras_df_train, keras_df_test = train_test_split(keras_df, test_size=0.33, random_state=42)

In [ ]:
pd.__version__

In [ ]:
mpl.__version__

In [ ]:
np.__version__

In [ ]:
sns.__version__

In [ ]:
def thresholding_algo_v2(
    y, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    #-----
    if len(y) < lag:
        return dict(signals = np.asarray(np.zeros(len(y))),
                    avgFilter = np.asarray(np.mean(y)),
                    stdFilter = np.asarray(np.std(y)))
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    non_signal_Y = []
    for i in range(lag, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = 0
            filteredY[i] = y[i]
            non_signal_Y.append(y[i])    
        else:
            if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter[i-1]:
                #-------------------------
                # If stdFilter[i-1]==0, try np.mean(stdFilter)
                # If both==0, use raw value, y[i] - avgFilter[i-1]
                if stdFilter[i-1]>0:
                    signals[i] = (y[i] - avgFilter[i-1])/stdFilter[i-1]
                else:
                    if np.mean(stdFilter)>0:
                        signals[i] = (y[i] - avgFilter[i-1])/np.mean(stdFilter)
                    else:
                        signals[i] = (y[i] - avgFilter[i-1])
                #-------------------------
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    filteredY[i] = influence * y[i] + (1 - influence) * np.mean(non_signal_Y)
            else:
                signals[i] = 0
                filteredY[i] = y[i]
                non_signal_Y.append(y[i])
        avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
        stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return np.asarray(signals)

In [ ]:
def thresholding_algo_v3(
    y, 
    lag, 
    threshold, 
    influence, 
    signal_abs_threshold=1.0
):
    #-----
    if len(y) < lag:
        return dict(signals = np.asarray(np.zeros(len(y))),
                    avgFilter = np.asarray(np.mean(y)),
                    stdFilter = np.asarray(np.std(y)))
    #-----
    signals = np.zeros(len(y))
    filteredY = np.array(y)
    avgFilter = [0]*len(y)
    stdFilter = [0]*len(y)
    avgFilter[lag - 1] = np.mean(y[0:lag])
    stdFilter[lag - 1] = np.std(y[0:lag])
    non_signal_Y = []
    for i in range(lag, len(y)):
        if y[i] < signal_abs_threshold:
            signals[i] = 0
            filteredY[i] = y[i]
            non_signal_Y.append(y[i])    
        else:
            if abs(y[i] - avgFilter[i-1]) > threshold * stdFilter[i-1]:
                signals[i]=1
                #-------------------------
                if len(non_signal_Y)==0:
                    filteredY[i] = influence * y[i]
                else:
                    filteredY[i] = influence * y[i] + (1 - influence) * np.mean(non_signal_Y)
            else:
                signals[i] = 0
                filteredY[i] = y[i]
                non_signal_Y.append(y[i])
        avgFilter[i] = np.mean(filteredY[(i-lag+1):i+1])
        stdFilter[i] = np.std(filteredY[(i-lag+1):i+1])

    return np.asarray(signals)

In [ ]:
def set_signal_in_df_i(
    df_i, 
    lag, 
    threshold,
    influence, 
    signal_abs_threshold, 
    value_col='mean_TRS value', 
    SN_col='serialnumber',
    signal_col='signals', 
    return_signal_group_col='signal_grp'
):
    r"""
    """
    #-------------------------
    assert(df_i[SN_col].nunique()==1)
    #-------------------------
    rtpd_i = thresholding_algo_v3(
        y=df_i[value_col].tolist(), 
        lag=lag,
        threshold=threshold, 
        influence=influence, 
        signal_abs_threshold=signal_abs_threshold
    )
    df_i[signal_col] = rtpd_i
    return df_i

In [ ]:
ami_df['serialnumber'].value_counts()

In [ ]:
dev_df = ami_df[ami_df['serialnumber'].isin(['794400462', '780269113', '764972211', '784554124', '762443496'])].copy()

In [ ]:
idk2 = dev_df.groupby(['serialnumber']).apply(
    lambda x: set_signal_in_df_i(
        df_i=x, 
        lag=48,
        threshold=5, 
        influence=0.0, 
        signal_abs_threshold=1.0, 
        value_col='value', 
    )
)

In [ ]:
dev_ami_df_evs = ami_df_evs.copy()
dev_ami_df_non = ami_df_non.copy()

In [ ]:
dev_ami_df_evs = dev_ami_df_evs.groupby(['serialnumber']).apply(
    lambda x: set_signal_in_df_i(
        df_i=x, 
        lag=48,
        threshold=5, 
        influence=0.0, 
        signal_abs_threshold=1.0, 
        value_col='value', 
    )
)

In [ ]:
dev_ami_df_non = dev_ami_df_non.groupby(['serialnumber']).apply(
    lambda x: set_signal_in_df_i(
        df_i=x, 
        lag=48,
        threshold=5, 
        influence=0.0, 
        signal_abs_threshold=1.0, 
        value_col='value', 
    )
)

In [ ]:
keras_df_evs = dev_ami_df_evs.reset_index(drop=True).groupby(['serialnumber']).apply(
    lambda x: make_it_funky(
        df_i=x, 
        n_entries=500, 
        val_col='signals', 
        SN_col='serialnumber'
    )
)
keras_df_non = dev_ami_df_non.reset_index(drop=True).groupby(['serialnumber']).apply(
    lambda x: make_it_funky(
        df_i=x, 
        n_entries=500, 
        val_col='signals', 
        SN_col='serialnumber'
    )
)

In [ ]:
keras_df_evs['target']=1
keras_df_non['target']=0

In [ ]:
keras_df = pd.concat([keras_df_evs, keras_df_non])

In [ ]:
keras_df

In [ ]:
keras_df=keras_df.dropna()

In [ ]:
keras_df

In [ ]:
keras_df_train, keras_df_test = train_test_split(keras_df, test_size=0.33, random_state=42)

In [ ]:
x_train = keras_df_train[[x for x in keras_df_train.columns.tolist() if x != 'target']].values
y_train = keras_df_train['target'].values
x_train=x_train.reshape((x_train.shape[0], x_train.shape[1], 1))

In [ ]:
x_test = keras_df_test[[x for x in keras_df_test.columns.tolist() if x != 'target']].values
y_test = keras_df_test['target'].values
x_test=x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [ ]:
model = make_model(input_shape=x_train.shape[1:])
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 500
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

In [ ]:
model = keras.models.load_model("best_model.h5")

test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test accuracy", test_acc)
print("Test loss", test_loss)

In [ ]:
train_loss, train_acc = model.evaluate(x_train, y_train)
print("Train accuracy", train_acc)
print("Train loss", train_loss)

In [ ]:
ami_df[ami_df['serialnumber']=='791084258']